In [1]:
import os.path as op
from os import makedirs
import numpy as np
from nilearn.image import load_img, new_img_like
from nilearn.datasets import fetch_atlas_aal
from ants import apply_transforms, image_read, image_write

# Define parameters here
bids="/path/to/nsd_bids/" # Path to BIDS root
fmriprep_dir = op.join("derivatives","fmriprep") # BIDS-relative path to fMRIPrep
fitlins_dir = op.join("derivatives","fitlins") # BIDS-relative path to Fitlins
subject = "sub-01" # Subject name
node_name = "sessionFloc6MP50ACompCor" # Node name defined by Fitlins model
contrast_name = "charactersGtOther" # Contrast name defined by Fitlins model
percentile = 90 # Find top 10% of voxels in left IFG
out_dir = op.join(bids, 'derivatives', 'threshold_fROIs', subject)
if not op.exists(out_dir):
    makedirs(out_dir)
hemi = "L"
region = "Fusiform"
aal_regionname = f"Fusiform_{hemi}" # Region to extract
out_path = op.join(out_dir, f"{subject}_hemi-{hemi}_space-T1w_contrast-{contrast_name}_stat-z_desc-{region}_desc-thresholded_roi.nii.gz") # Where to save the output image

## Load Z-stat image
z_img_path = op.join(bids,fitlins_dir,f"node-{node_name}",subject,
                          f"{subject}_contrast-{contrast_name}_stat-z_statmap.nii.gz")
z_img = load_img(z_img_path)
z_img_affine = z_img.affine # Affine matrix
z_img_values = z_img.get_fdata() # Array of data

In [2]:
### Register the AAL atlas to native space using ANTs

# Load the Z-stat img in a way that ANTs likes
z_img_ants = image_read(z_img_path)

# Download the AAL atlas
AAL = fetch_atlas_aal()
AAL_MNI_path = AAL.maps
AAL_MNI = image_read(AAL_MNI_path)

# Find the MNI-to-Native space transformation from fMRIPrep
MNI2Native_reg = op.join(bids,fmriprep_dir,subject,"anat",f"{subject}_from-MNI152NLin2009cAsym_to-T1w_mode-image_xfm.h5")

# Run the registration and save out the image
AAL_native_path = op.join(bids,fmriprep_dir,subject,"anat",f"{subject}_desc-AAL_dseg.nii.gz") # Where to save registered atlas
AAL_native = apply_transforms(z_img_ants, AAL_MNI, transformlist=[MNI2Native_reg], interpolator="nearestNeighbor")
image_write(AAL_native, AAL_native_path)

In [3]:
### Get binary mask of the region

# Get index of AAL labels that contains "Fusiform_L"
region_index = AAL.labels.index(aal_regionname)
region_value = np.asarray(AAL.indices)[region_index].astype(int)
segmentation = load_img(AAL_native_path) # Load out native space segmentation
segmentation_values = segmentation.get_fdata()
# Make binary mask of all region values
region_mask_vals = np.full(np.shape(segmentation_values), False)
region_mask_vals[segmentation_values==region_value] = True
region_mask = new_img_like(z_img, region_mask_vals, affine=z_img_affine, copy_header=True)

In [4]:
### Find top 10% of values in region
z_img_values_masked = z_img_values[region_mask_vals]
percentile_thresh = np.percentile(z_img_values_masked, percentile) # The critical threshold
z_inds_gt_thresh_in_mask = (z_img_values>=percentile_thresh) * region_mask_vals
z_img_masked_thresholded = new_img_like(z_img, z_inds_gt_thresh_in_mask, affine=z_img_affine, copy_header=True)
z_img_masked_thresholded.to_filename(out_path) # Save the image out